# SnorkelMED - Identifying Opioid-Induced Respiratory Depression  

The purpose of this analysis is to probabilistically identify which patient visits included an opioid-induced respiratory depression (OIRD) event. 

In [ ]:
%matplotlib inline
import os
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

from snorkel.labeling import labeling_function, PandasLFApplier, LFAnalysis
from snorkel.labeling.model import MajorityLabelVoter, LabelModel
from snorkel.analysis import get_label_buckets

majority_model = MajorityLabelVoter()
label_model = LabelModel(cardinality=2, verbose=True)

import helper as hlp
import importlib
importlib.reload(hlp)

# global variables
ABSTAIN = -1; CONTROL = 0; CASE = 1

In [ ]:
# load updated training/dev/valid data after labeling from previous round
df_train, df_dev, df_valid, df_test = hlp.load_data(round=2)

# re-attach numeric data to reflect any updated rules
df_train, df_dev = hlp.reattach_numeric_data(df_train, df_dev)

# keep confounding diagnoses visits available
confounding_diagnosis_present = pd.read_csv('../sd_structured/icd/visits_with_confounding_icd_codes.csv')

In [ ]:
# given the previous changes to excluding persons who are also in the test set,
#   ensure no persons in the test set are in the train, valid, or dev sets
assert sum(df_train['person_id'].isin(df_test['person_id'])) == 0
assert sum(df_dev['person_id'].isin(df_test['person_id'])) == 0
assert sum(df_valid['person_id'].isin(df_test['person_id'])) == 0

In [ ]:
# store Y values for ease of evaluation
Y_dev = df_dev['label'].values
Y_dev = np.where(Y_dev=='case', 1, 0) 

#Y_valid = df_valid['label'].values

# Round 2 - Add More Learning Functions

In [ ]:
# create dictionary to keep track of rule names for easier reference later
lfd = dict()

@labeling_function()
def LF_naloxone_admin(x):
    if x['naloxone_admin_prob'] >= 0.75:
        if x['counts_naloxone_effective'] > 0:
            return CASE
        elif x['counts_naloxone_NOT_effective'] > 0:
            return CONTROL
        else:
            return CASE
    elif x['naloxone_admin_prob'] < 0.75:
        return CONTROL
    else:
        # if missing
        return ABSTAIN
lfd['LF_naloxone_admin'] = 0
    
@labeling_function()
def LF_respiratory_failure_any(x):
    # this ended up being a heavily-weighted rule to lean controls toward cases
    # but it has excellent coverage
    # so adding logic checks to see if other good rules might suggest non-opioid causes
    # I thought snorkel should handle all these interactions, but I'm not seeing that to be the case.
    if '1' in x['respiratory_failure_any'].lower(): 
        # if there is a confounding diagonsis that would make respiratory failure likely
        if x['visit_occurrence_id'] in confounding_diagnosis_present['visit_occurrence_id'].unique():
            return CONTROL
        # or if there is a lack of any other evidence to suggest the patient is a case
        elif x['counts_naloxone'] == 0 and \
             np.isnan(x['naloxone_admin_prob']) and \
             x['counts_altered_mental_status'] == 0 and \
             x['counts_narcotic_overdose'] == 0 and \
             x['counts_hypoxia'] == 0 and \
             x['counts_decrease_opioids'] == 0:
            return CONTROL
        else:
            return CASE
    return CONTROL
lfd['LF_respiratory_failure_any'] = max(lfd.values()) + 1

@labeling_function()
def LF_eligible_vent(x):
    if 'yes' in x['eligible_vent'].lower(): 
        return CASE
    return ABSTAIN
lfd['LF_eligible_vent'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_naloxone(x):
    if x['counts_naloxone'] > 0: 
        return CASE
    return ABSTAIN
lfd['LF_counts_naloxone'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_resp_care_notes(x):
    if x['counts_resp_care_notes'] == 0:
        return CONTROL
    return ABSTAIN
lfd['LF_counts_resp_care_notes'] = max(lfd.values()) + 1

#@labeling_function()
#def LF_counts_rapid_response(x):
    # in Round 2, moving this rule to the competing diagnosis rule
#    if x['counts_rapid_response'] == 0:
#        return CONTROL
#    elif x['counts_rapid_response'] == 1:
#        return ABSTAIN
#    return CASE
#lfd['LF_counts_rapid_response'] = max(lfd.values()) + 1

In [ ]:
# Round 2 Additions

@labeling_function()
def LF_counts_no_acute_events(x):
    # this ended up being a heavily-weighted rule to lean cases toward controls
    # so additional logic checks to see if other "acute" things happened during the visit
    #   (even if they didn't happen for several days where this was recorded)
    if x['counts_no_acute_events'] > 0:
        if x['counts_naloxone'] + x['counts_rapid_response'] + x['counts_altered_mental_status'] + \
            x['counts_narcotic_overdose'] + x['counts_hypoxia'] > 0:
            return ABSTAIN
        return CONTROL
    return ABSTAIN
lfd['LF_counts_no_acute_events'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_altered_mental_status(x):
    if x['counts_altered_mental_status'] > 0:
        return CASE
    return ABSTAIN
lfd['LF_counts_altered_mental_status'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_narcotic_overdose(x):
    if x['counts_narcotic_overdose'] > 0:
        return CASE
    return ABSTAIN
lfd['LF_counts_narcotic_overdose'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_hypoxia(x):
    if x['counts_hypoxia'] > 0:
        return CASE
    return ABSTAIN
lfd['LF_counts_hypoxia'] = max(lfd.values()) + 1

@labeling_function()
def LF_counts_decrease_opioids(x):
    if x['counts_decrease_opioids'] > 0:
        return CASE
    return ABSTAIN
lfd['LF_counts_decrease_opioids'] = max(lfd.values()) + 1

# the following 2 labeling functions have been moved to the 
#   naloxone_admin probability labeling function
#@labeling_function()
#def LF_counts_naloxone_effective(x):
#    if x['counts_naloxone_effective'] > 0:
#        return CASE
#    return ABSTAIN
#lfd['LF_counts_naloxone_effective'] = max(lfd.values()) + 1

#@labeling_function()
#def LF_counts_naloxone_NOT_effective(x):
#    if x['counts_naloxone_NOT_effective'] > 0:
#        return CONTROL
#    return ABSTAIN
#lfd['LF_counts_naloxone_NOT_effective'] = max(lfd.values()) + 1

@labeling_function()
def LF_confounding_diagnosis_for_rrt(x):
    if x['visit_occurrence_id'] in confounding_diagnosis_present['visit_occurrence_id'].unique(): 
        return CONTROL
    elif x['counts_rapid_response'] > 0:
        return CASE
    return ABSTAIN
lfd['LF_confounding_diagnosis_for_rrt'] = max(lfd.values()) + 1

@labeling_function()
def LF_extended_vent_time(x):
    # if on the vent for 4 or more days
    if 'yes;;yes;;yes;;yes' in x['eligible_vent'].lower(): 
        return CONTROL
    return ABSTAIN
lfd['LF_extended_vent_time'] = max(lfd.values()) + 1

In [ ]:
# combine all relevant LFs
lfs = [LF_naloxone_admin,
      LF_respiratory_failure_any,
      LF_eligible_vent,
      LF_counts_naloxone,
      LF_counts_resp_care_notes,
      #LF_counts_rapid_response,
      LF_counts_no_acute_events,
      LF_counts_altered_mental_status,
      LF_counts_narcotic_overdose,
      LF_counts_hypoxia,
      LF_counts_decrease_opioids,
      #LF_counts_naloxone_effective,
      #LF_counts_naloxone_NOT_effective,
      LF_confounding_diagnosis_for_rrt,
      LF_extended_vent_time,
      ]

# apply LFs
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_dev = applier.apply(df=df_dev)

In [ ]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

In [ ]:
LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=Y_dev)

### Further Exploration

In [ ]:
df_train.iloc[L_train[:, lfd['LF_respiratory_failure_any']] == CONTROL].head()

### Conflicts

In [ ]:
buckets = get_label_buckets(L_dev[:, lfd['LF_confounding_diagnosis_for_rrt']], 
                            L_dev[:, lfd['LF_counts_no_acute_events']])
df_dev[['label', 'counts_no_acute_events', 'counts_rapid_response', 'sepsis', 'cva']] \
    .iloc[buckets[(CONTROL, CONTROL)]]

`eligible_vent` had low covered & low empirical accuracy, and since the introduction of `extended_vent_time` (which has higher empirical accuracy), I'm throwing out `eligible_vent`  

`altered_mental_status` has good coverage but poor empirical accuracy. Particularly now that we have the `confounding_diagnosis_present` rule, we could either remove or even consider incorporating it under the diagnosis rule. 

In [ ]:
lfs.remove(LF_eligible_vent)
lfs.remove(LF_counts_altered_mental_status)
lfs

In [ ]:
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_dev = applier.apply(df=df_dev)

## Voting

In [ ]:
# ensure the code can run
label_model.fit(L_train=L_train, Y_dev = Y_dev, 
                n_epochs = 4000, lr = 0.004, #l2 = 0.01,
                optimizer = 'adamax', lr_scheduler = 'step', #prec_init = 0.7,
                log_freq = 100, seed = 987)
analysis = LFAnalysis(L=L_train, lfs=lfs).lf_summary(est_weights=label_model.get_weights())

### Hyper-Parameter Tuning

In [ ]:
# create empty dataframe to hold learned weights for each rule
df_cols = ['n_epochs', 'lr', 'lr_scheduler']
df_cols.extend(analysis.index)

# specify potential hyperparameters
n_epochs = [2000, 4000]
lr = [0.001, 0.005, 0.01]
lr_scheduler = ['step', 'exponential', 'linear']

# tune - in round 2, started adding new seeds also (e.g., 987, 456, & 123)
df_tune, df_tune_long = hlp.label_model_tuning(lfs, df_cols, 
                                               L_train, L_dev, Y_dev, 
                                               n_epochs, lr, lr_scheduler,
                                               seed = 123)

In [ ]:
# review best accuracies among development set
df_tune[df_tune['accuracy'] == np.max(df_tune['accuracy'])]

In [ ]:
for scheduler in lr_scheduler:
    g = sns.FacetGrid(df_tune_long[df_tune_long['lr_scheduler']==scheduler], 
                      col='lr', hue='learning_function', col_wrap=3, height=4)
    g = (g.map(plt.scatter, 'n_epochs', 'learned_weight')
            .add_legend()
            .fig.suptitle('Learned Weights Using ' + str(scheduler) + ' Scheduler', 
                          y=1.05, fontsize=16))

In [ ]:
label_model.fit(L_train=L_train, Y_dev = Y_dev, n_epochs = 2000, lr = 0.005, optimizer = 'adamax', 
                lr_scheduler = 'exponential', log_freq = 100, seed = 123)

In [ ]:
LFAnalysis(L=L_train, lfs=lfs) \
    .lf_summary(est_weights = label_model.get_weights()) \
    .sort_values(by='Learned Weight', ascending=False)

In [ ]:
majority_acc = majority_model.score(L=L_dev, Y=Y_dev)["accuracy"]
print(f"{'Majority Vote Accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_dev, Y=Y_dev)["accuracy"]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

In [ ]:
# assign probabilities from either majority vote or label model
#gen_probs_train = majority_model.predict_proba(L=L_train)
#gen_probs_dev = majority_model.predict_proba(L=L_dev)

gen_probs_train = label_model.predict_proba(L=L_train)
gen_probs_dev = label_model.predict_proba(L=L_dev)

In [ ]:
hlp.plot_probabilities_histogram(gen_probs_train[:, CASE])

In [ ]:
hlp.plot_probabilities_histogram(gen_probs_dev[:, CASE])

In [ ]:
dev_with_probs = df_dev.copy()
dev_with_probs['label_model_prob'] = gen_probs_dev[:, CASE]

cases_low_prob = dev_with_probs[(dev_with_probs['label']=='case') & \
                                (dev_with_probs['label_model_prob']<0.5)]
controls_high_prob = dev_with_probs[(dev_with_probs['label']=='control') & \
                                    (dev_with_probs['label_model_prob']>0.5)]

cases_low_prob \
    [['label', 'label_model_prob', 'naloxone_admin_prob', 'eligible_vent', 'respiratory_failure_any',
     'counts_naloxone', 'counts_resp_care_notes', 'counts_rapid_response', 
     'counts_no_acute_events', 'counts_altered_mental_status', 'counts_narcotic_overdose',
     'counts_hypoxia','counts_decrease_opioids', 'counts_naloxone_effective', 'counts_naloxone_NOT_effective',
     'sepsis', 'cva']]

In [ ]:
lfs

In [ ]:
L_dev[cases_low_prob.index.values]

In [ ]:
controls_high_prob \
    [['label', 'label_model_prob', 'naloxone_admin_prob', 'eligible_vent', 'respiratory_failure_any',
     'counts_naloxone', 'counts_resp_care_notes', 'counts_rapid_response', 
     'counts_no_acute_events', 'counts_altered_mental_status', 'counts_narcotic_overdose',
     'counts_hypoxia','counts_decrease_opioids', 'counts_naloxone_effective', 'counts_naloxone_NOT_effective',
     'sepsis', 'cva']]

In [ ]:
L_dev[controls_high_prob.index.values]

In [ ]:
# exploration of associations 
from matplotlib  import cm

df_fig = dev_with_probs
colors = ['red' if p=='case' else 'blue' for p in df_fig['label']]
plt.scatter(df_fig['naloxone_admin_prob'],
            df_fig['label_model_prob'],
            c=colors)

In [ ]:
plt.hist(dev_with_probs[dev_with_probs['label']=='case']['label_model_prob'], bins=20)

In [ ]:
plt.hist(dev_with_probs[dev_with_probs['label']=='control']['label_model_prob'], bins=20)

Currently, it looks like the dev set is illustrating really nice performance on controls with all probabilities for controls being < 7%. Most of the cases have very high probabilities, but there are still 2 patients in the cases group who have *very* low scores. Notably, when I look at the manual review for those 2 patients, these were not the most confident cases. 

In [ ]:
# attach LabelModel predictions to dataframes
train_with_probs = df_train.copy()
train_with_probs['label_model_prob'] = gen_probs_train[:, 1]

In [ ]:
# extract top 20 highest probabilities from train set
top_probs = train_with_probs.nlargest(n=20, columns='label_model_prob')

# send half to the dev set & half to the valid set
visits_for_dev = top_probs['visit_occurrence_id'].sample(frac=0.5, random_state=123)
visits_for_valid = top_probs[~np.isin(top_probs['visit_occurrence_id'], visits_for_dev)]['visit_occurrence_id']

# concatenate the respective training set rows to dev & valid sets
df_dev3 = pd.concat([df_dev, df_train[df_train['visit_occurrence_id'].isin(visits_for_dev)]], sort=True)
df_valid3 = pd.concat([df_valid, df_train[np.isin(df_train['visit_occurrence_id'], visits_for_valid)]], sort=True)

# remove the rows from the training set
df_train3 = df_train.drop(top_probs.index)

assert df_dev3.shape[0] == df_dev.shape[0] + 0.5*top_probs.shape[0]
assert df_valid3.shape[0] == df_valid.shape[0] + 0.5*top_probs.shape[0]
assert df_train3.shape[0] == df_train.shape[0] - top_probs.shape[0]
assert not np.isin(df_train3['visit_occurrence_id'], df_dev3['visit_occurrence_id']).any()
assert not np.isin(df_train3['visit_occurrence_id'], df_valid3['visit_occurrence_id']).any()

In [ ]:
# export for manual review
#df_train3.to_csv('./train_set3.csv', index=False)
#df_dev3.to_csv('./dev_set3.csv', index=False)
#df_valid3.to_csv('./valid_set3.csv', index=False)